In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier, Pool
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, roc_curve
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from scipy.stats import uniform, randint
import optuna
import os
from pathlib import Path

# --- Загрузка данных ---
file_url_pca = 'https://drive.google.com/uc?export=download&id=1SuUhkpfj-3uJQnxwmUCyDUogfa2TixTe'
file_url_manual = 'https://drive.google.com/uc?export=download&id=1p8VYp23oOylSFrfJztQVheNLop-bX40o'

df_pca = pd.read_csv(file_url_pca, encoding='utf-8')
df_manual = pd.read_csv(file_url_manual, encoding='utf-8')

# --- Определяем фактические целевые переменные, которые уже логарифмированы ---
# Поскольку вы подтвердили, что 'IC50, mM', 'CC50, mM', 'SI' уже логарифмированы,
# мы будем использовать их напрямую как наши "лог-цели".
TARGETS_ACTUAL_LOGGED = ['CC50, mM']

print("PCA Data (actual logged targets):")
print(df_pca[TARGETS_ACTUAL_LOGGED].head())
print("\nManual Data (actual logged targets):")
print(df_manual[TARGETS_ACTUAL_LOGGED].head())

# --- Создание бинарных целевых переменных для классификации ---

classification_targets = {}

# 2. CC50 > медианы
median_cc50_pca = df_pca['CC50, mM'].median()
df_pca['is_CC50_above_median'] = (df_pca['CC50, mM'] > median_cc50_pca).astype(int)
median_cc50_manual = df_manual['CC50, mM'].median()
df_manual['is_CC50_above_median'] = (df_manual['CC50, mM'] > median_cc50_manual).astype(int)
classification_targets['is_CC50_above_median'] = 'CC50, mM'


print("\nСозданные бинарные целевые переменные:")
print("PCA - is_CC50_above_median value counts:\n", df_pca['is_CC50_above_median'].value_counts())

print("Manual - is_CC50_above_median value counts:\n", df_manual['is_CC50_above_median'].value_counts())

# --- Вспомогательная функция для расчета метрик классификации ---
def calculate_classification_metrics(y_true, y_pred, y_pred_proba):
    """Вычисляет метрики классификации: Accuracy, Precision, Recall, F1, ROC-AUC."""
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, zero_division=0) # Добавлено zero_division
    recall = recall_score(y_true, y_pred, zero_division=0)
    f1 = f1_score(y_true, y_pred, zero_division=0)
    roc_auc = roc_auc_score(y_true, y_pred_proba)
    return accuracy, precision, recall, f1, roc_auc

# --- Функции для каждого метода оптимизации (адаптированные для классификации) ---

def run_randomized_search_classifier(model_instance, param_distributions, X_train_scaled, y_train, n_iter_search=20):
    """Выполняет RandomizedSearchCV для подбора гиперпараметров для классификации."""
    if not param_distributions:
        model_instance.fit(X_train_scaled, y_train)
        return model_instance, {}

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    random_search = RandomizedSearchCV(model_instance, param_distributions, n_iter=n_iter_search,
                                       cv=cv, scoring='roc_auc',
                                       n_jobs=-1, verbose=0, random_state=42)
    random_search.fit(X_train_scaled, y_train)
    return random_search.best_estimator_, random_search.best_params_

def run_grid_search_classifier(model_instance, param_grid, X_train_scaled, y_train):
    """Выполняет GridSearchCV для подбора гиперпараметров для классификации."""
    if not param_grid:
        model_instance.fit(X_train_scaled, y_train)
        return model_instance, {}

    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    grid_search = GridSearchCV(model_instance, param_grid, cv=cv, scoring='roc_auc',
                               n_jobs=-1, verbose=0)
    grid_search.fit(X_train_scaled, y_train)
    return grid_search.best_estimator_, grid_search.best_params_

def run_optuna_search_classifier(model_class, optuna_search_space, X_train_scaled, y_train, n_trials=20):
    """Выполняет оптимизацию гиперпараметров с помощью Optuna для классификации."""
    def objective(trial):
        params = optuna_search_space(trial)

        # Обработка random_state/random_seed для Optuna
        # Random_state может быть не поддерживаем для всех моделей или определенных solvers
        # Здесь мы исходим из того, что Optuna space уже определяет правильный параметр ('random_state' или 'random_seed')
        if model_class in [LogisticRegression, MLPClassifier] and 'random_state' in params:
            model = model_class(**{k: v for k, v in params.items() if k != 'random_state'})
        else:
            model = model_class(**params)

        kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        cv_scores = []
        for train_idx, val_idx in kf.split(X_train_scaled, y_train):
            X_train_fold, X_val_fold = X_train_scaled[train_idx], X_train_scaled[val_idx]
            y_train_fold, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]

            try:
                if isinstance(model, CatBoostClassifier):
                    train_pool = Pool(X_train_fold, y_train_fold)
                    val_pool = Pool(X_val_fold, y_val_fold)
                    model.fit(train_pool, eval_set=val_pool, early_stopping_rounds=10, verbose=False)
                    y_val_pred_proba = model.predict_proba(X_val_fold)[:, 1]
                else:
                    model.fit(X_train_fold, y_train_fold)
                    y_val_pred_proba = model.predict_proba(X_val_fold)[:, 1]

                roc_auc_fold = roc_auc_score(y_val_fold, y_val_pred_proba)
                cv_scores.append(roc_auc_fold)
            except Exception as e:
                # print(f"Ошибка при обучении/предсказании в Optuna (фолд): {e}") # Для дебага
                return -float('inf')

        return -np.mean(cv_scores)

    study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=42))
    study.optimize(objective, n_trials=n_trials, show_progress_bar=False, catch=(ValueError, Exception))

    best_params = study.best_params
    
    # Final model instance with best parameters
    if model_class in [LogisticRegression, MLPClassifier] and 'random_state' in best_params:
        best_model_instance = model_class(**{k: v for k, v in best_params.items() if k != 'random_state'})
    else:
        best_model_instance = model_class(**best_params)

    try:
        if isinstance(best_model_instance, CatBoostClassifier):
            train_pool_final = Pool(X_train_scaled, y_train)
            best_model_instance.fit(train_pool_final, verbose=False)
        else:
            best_model_instance.fit(X_train_scaled, y_train)
    except Exception as e:
        # print(f"Ошибка при окончательном обучении CatBoost: {e}") # Для дебага
        return None, {}

    return best_model_instance, best_params

# --- Общая функция для оценки моделей с различными оптимизаторами (адаптированная) ---
def evaluate_model_with_optimizer_classifier(model_name, model_class, params_config, X, y, target_name, optimizer_type):
    """Оценивает производительность модели классификации, используя указанный метод оптимизации."""

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    best_model = None
    best_params = {}

    # Инициализация параметров для воспроизводимости:
    model_init_params = {}
    if model_name == "CatBoostClassifier":
        model_init_params['random_seed'] = 42
    elif model_name in ["LogisticRegression", "RandomForestClassifier", "XGBClassifier", "MLPClassifier"]:
        # Эти модели обычно принимают random_state для воспроизводимости
        model_init_params['random_state'] = 42

    if optimizer_type == 'RandomizedSearchCV':
        param_distributions = params_config.get('random_dist', {})
        # Для LogisticRegression, если нет dist, используем дефолтный инстанс
        if model_name == "LogisticRegression" and not param_distributions:
             model_instance = model_class(**model_init_params)
             model_instance.fit(X_train_scaled, y_train)
             best_model, best_params = model_instance, {}
        else:
            best_model, best_params = run_randomized_search_classifier(model_class(**model_init_params), param_distributions, X_train_scaled, y_train, n_iter_search=20)

    elif optimizer_type == 'GridSearchCV':
        param_grid = params_config.get('grid_params', {})
        if not param_grid:
            model_instance = model_class(**model_init_params)
            model_instance.fit(X_train_scaled, y_train)
            best_model, best_params = model_instance, {}
        else:
            best_model, best_params = run_grid_search_classifier(model_class(**model_init_params), param_grid, X_train_scaled, y_train)

    elif optimizer_type == 'Optuna':
        optuna_space = params_config.get('optuna_space')
        if optuna_space is None:
            model_instance = model_class(**model_init_params)
            model_instance.fit(X_train_scaled, y_train)
            best_model, best_params = model_instance, {}
        else:
            # Optuna уже обрабатывает random_state/random_seed в своей objective функции
            best_model, best_params = run_optuna_search_classifier(model_class, optuna_space, X_train_scaled, y_train, n_trials=20)
    else:
        raise ValueError(f"Неизвестный тип оптимизатора: {optimizer_type}")

    if best_model is None:
        return None

    y_pred = best_model.predict(X_test_scaled)
    # predict_proba может отсутствовать для некоторых моделей (например, SVM с probability=False)
    # или если модель не была обучена с этой функциональностью.
    # Проверяем наличие predict_proba
    if hasattr(best_model, "predict_proba") and len(best_model.predict_proba(X_test_scaled).shape) > 1:
        y_pred_proba = best_model.predict_proba(X_test_scaled)[:, 1]
    else:
        # Для моделей без predict_proba, ROC-AUC не может быть рассчитан.
        # В таком случае, можно либо пропустить ROC-AUC, либо вернуть NaN.
        # Для SVM, если probability=True не установлен при инициализации, его не будет.
        # Для LogisticRegression и Tree-based моделей predict_proba всегда есть.
        print(f"Warning: Model {model_name} does not have predict_proba or it's not applicable. ROC-AUC will be NaN.")
        y_pred_proba = np.full_like(y_pred, np.nan, dtype=float) # Заполняем NaN для ROC-AUC

    accuracy, precision, recall, f1, roc_auc = calculate_classification_metrics(y_test, y_pred, y_pred_proba)

    return {
        'model': model_name,
        'optimizer': optimizer_type,
        'target': target_name,
        'best_params': best_params,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'roc_auc': roc_auc
    }

# --- Определение моделей и их гиперпараметров для разных оптимизаторов (адаптированные для классификации) ---
models_config_classifier = {
    "LogisticRegression": {
        "class": LogisticRegression,
        "random_dist": {
            'C': uniform(loc=0.1, scale=10),
            'penalty': ['l1', 'l2'],
            'solver': ['liblinear']
        },
        "grid_params": {
            'C': [0.1, 1.0, 10.0],
            'penalty': ['l1', 'l2'],
            'solver': ['liblinear']
        },
        "optuna_space": lambda trial: {
            'C': trial.suggest_float('C', 0.1, 10.0, log=True),
            'penalty': trial.suggest_categorical('penalty', ['l1', 'l2']),
            'solver': 'liblinear',
            'random_state': 42 # Добавлен random_state здесь, чтобы управлять им
        }
    },
    "RandomForestClassifier": {
        "class": RandomForestClassifier,
        "random_dist": {
            'n_estimators': randint(50, 200),
            'max_depth': [5, 10, None],
            'min_samples_split': randint(2, 8)
        },
        "grid_params": {
            'n_estimators': [100, 150],
            'max_depth': [5, 10],
        },
        "optuna_space": lambda trial: {
            'n_estimators': trial.suggest_int('n_estimators', 50, 200),
            'max_depth': trial.suggest_categorical('max_depth', [5, 10, 15, None]),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 8),
            'random_state': 42
        }
    },
    "XGBClassifier": {
        "class": XGBClassifier,
        "random_dist": {
            'n_estimators': randint(50, 200),
            'learning_rate': uniform(0.01, 0.15),
            'max_depth': randint(3, 8),
            'subsample': uniform(0.7, 0.3),
            'use_label_encoder': [False]
        },
        "grid_params": {
            'n_estimators': [100, 150],
            'learning_rate': [0.05, 0.1],
            'max_depth': [3, 5],
            'use_label_encoder': [False]
        },
        "optuna_space": lambda trial: {
            'n_estimators': trial.suggest_int('n_estimators', 50, 200),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.15, log=True),
            'max_depth': trial.suggest_int('max_depth', 3, 8),
            'subsample': trial.suggest_float('subsample', 0.7, 1.0),
            'eval_metric': 'logloss',
            'n_jobs': -1,
            'random_state': 42,
            'use_label_encoder': False
        }
    },
    "CatBoostClassifier": {
        "class": CatBoostClassifier,
        "random_dist": {
            'iterations': randint(50, 200),
            'learning_rate': uniform(0.01, 0.15),
            'depth': randint(3, 8),
            'l2_leaf_reg': uniform(1, 7)
        },
        "grid_params": {
            'iterations': [100, 150],
            'learning_rate': [0.05, 0.1],
            'depth': [3, 5]
        },
        "optuna_space": lambda trial: {
            'iterations': trial.suggest_int('iterations', 50, 200),
            'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.15, log=True),
            'depth': trial.suggest_int('depth', 3, 8),
            'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-2, 10, log=True),
            'verbose': False,
            'random_seed': 42, # CatBoost uses random_seed
            'thread_count': -1,
            'objective': 'Logloss'
        }
    },
    "MLPClassifier": {
        "class": MLPClassifier,
        "random_dist": {
            'hidden_layer_sizes': [(50,), (100,), (50, 50), (100, 50)],
            'alpha': uniform(0.0001, 0.005),
            'learning_rate_init': uniform(0.0001, 0.005)
        },
        "grid_params": {
            'hidden_layer_sizes': [(50,), (100,)],
            'alpha': [0.0001, 0.001]
        },
        "optuna_space": lambda trial: {
            'hidden_layer_sizes': trial.suggest_categorical('hidden_layer_sizes', [(50,), (100,), (50, 50), (100, 50)]),
            'alpha': trial.suggest_float('alpha', 1e-5, 1e-2, log=True),
            'learning_rate_init': trial.suggest_float('learning_rate_init', 1e-4, 1e-2, log=True),
            'max_iter': 2000,
            'random_state': 42,
            'solver': 'adam'
        }
    }
}



PCA Data (actual logged targets):
   CC50, mM
0  5.173221
1  1.856738
2  5.088474
3  4.690023
4  4.943576

Manual Data (actual logged targets):
   CC50, mM
0  5.173221
1  1.856738
2  5.088474
3  4.690023
4  4.943576

Созданные бинарные целевые переменные:
PCA - is_CC50_above_median value counts:
 is_CC50_above_median
0    502
1    499
Name: count, dtype: int64
Manual - is_CC50_above_median value counts:
 is_CC50_above_median
0    502
1    499
Name: count, dtype: int64


In [4]:
# --- Основной цикл оценки с тремя методами оптимизации ---
all_classification_results = []
optimizers = ['RandomizedSearchCV', 'GridSearchCV', 'Optuna']

# Извлекаем признаки, исключая все целевые переменные (теперь просто TARGETS_ACTUAL_LOGGED)
# и новые бинарные целевые переменные.
columns_to_drop_common = TARGETS_ACTUAL_LOGGED + list(classification_targets.keys())

# Добавляем специфические для датасетов столбцы, которые не являются признаками (например, SMILES)
if 'SMILES' in df_pca.columns:
    columns_to_drop_pca_final = columns_to_drop_common + ['SMILES']
else:
    columns_to_drop_pca_final = columns_to_drop_common

if 'SMILES' in df_manual.columns:
    columns_to_drop_manual_final = columns_to_drop_common + ['SMILES']
else:
    columns_to_drop_manual_final = columns_to_drop_common

X_pca_features = df_pca.drop(columns=columns_to_drop_pca_final, errors='ignore')
X_manual_features = df_manual.drop(columns=columns_to_drop_manual_final, errors='ignore')


print("Начинаем процесс обучения и оценки моделей классификации...")

for target_name_classification in tqdm(classification_targets.keys(), desc="Прогнозирование задач классификации"):
    for data_source_name, X_data_features, df_data in [("PCA Aggregated", X_pca_features, df_pca), ("Manual Aggregated", X_manual_features, df_manual)]:
        y_data_classification = df_data[target_name_classification]

        num_models_to_run = 0
        for model_name, config in models_config_classifier.items():
            for optimizer_type in optimizers:
                # Уточненная логика для подсчета моделей
                if (optimizer_type == 'RandomizedSearchCV' and not config.get('random_dist', {})) and model_name != "LogisticRegression":
                    continue
                if (optimizer_type == 'GridSearchCV' and not config.get('grid_params', {})) and model_name != "LogisticRegression":
                    continue
                if (optimizer_type == 'Optuna' and config.get('optuna_space') is None) and model_name != "LogisticRegression":
                    continue
                num_models_to_run += 1

        with tqdm(total=num_models_to_run, desc=f"Оптимизация для {target_name_classification} ({data_source_name})", leave=False) as pbar_inner:
            for optimizer_type in optimizers:
                for model_name, config in models_config_classifier.items():
                    # Пропускаем неподходящие комбинации модель-оптимизатор, чтобы избежать ошибок и не тратить время
                    if (optimizer_type == 'RandomizedSearchCV' and not config.get('random_dist', {})) and model_name != "LogisticRegression":
                        pbar_inner.update(1)
                        continue
                    if (optimizer_type == 'GridSearchCV' and not config.get('grid_params', {})) and model_name != "LogisticRegression":
                        pbar_inner.update(1)
                        continue
                    if (optimizer_type == 'Optuna' and config.get('optuna_space') is None) and model_name != "LogisticRegression":
                        pbar_inner.update(1)
                        continue

                    model_class = config["class"]
                    params_config = config

                    pbar_inner.set_description(f"Оптимизация для {target_name_classification} ({data_source_name}) - {model_name} ({optimizer_type})")

                    result = evaluate_model_with_optimizer_classifier(model_name, model_class, params_config,
                                                                      X_data_features, y_data_classification, target_name_classification, optimizer_type)
                    if result:
                        result['data_source'] = data_source_name
                        all_classification_results.append(result)
                    pbar_inner.update(1)



Начинаем процесс обучения и оценки моделей классификации...


Прогнозирование задач классификации:   0%|          | 0/1 [00:00<?, ?it/s]

Оптимизация для is_CC50_above_median (PCA Aggregated):   0%|          | 0/15 [00:00<?, ?it/s]

C:\Users\user\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [19:58:40] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


0:	learn: 0.6457219	total: 141ms	remaining: 25.4s
1:	learn: 0.5848056	total: 144ms	remaining: 12.9s
2:	learn: 0.5237114	total: 150ms	remaining: 8.9s
3:	learn: 0.4753648	total: 154ms	remaining: 6.8s
4:	learn: 0.4372109	total: 158ms	remaining: 5.58s
5:	learn: 0.3983138	total: 162ms	remaining: 4.74s
6:	learn: 0.3842155	total: 166ms	remaining: 4.12s
7:	learn: 0.3578239	total: 169ms	remaining: 3.66s
8:	learn: 0.3416767	total: 173ms	remaining: 3.31s
9:	learn: 0.3224828	total: 178ms	remaining: 3.05s
10:	learn: 0.3088873	total: 183ms	remaining: 2.83s
11:	learn: 0.3012176	total: 189ms	remaining: 2.66s
12:	learn: 0.2876396	total: 192ms	remaining: 2.48s
13:	learn: 0.2664275	total: 197ms	remaining: 2.35s
14:	learn: 0.2539816	total: 203ms	remaining: 2.24s
15:	learn: 0.2438903	total: 206ms	remaining: 2.13s
16:	learn: 0.2399028	total: 211ms	remaining: 2.04s
17:	learn: 0.2347109	total: 217ms	remaining: 1.96s
18:	learn: 0.2307209	total: 221ms	remaining: 1.89s
19:	learn: 0.2227009	total: 224ms	remaining

...

C:\Users\user\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [20:10:17] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


0:	learn: 0.6025484	total: 5.4ms	remaining: 982ms
1:	learn: 0.5357384	total: 10.9ms	remaining: 986ms
2:	learn: 0.4866905	total: 16.7ms	remaining: 999ms
3:	learn: 0.4460188	total: 23.3ms	remaining: 1.04s
4:	learn: 0.4203270	total: 28.3ms	remaining: 1.01s
5:	learn: 0.4073395	total: 33.4ms	remaining: 985ms
6:	learn: 0.3722189	total: 39.8ms	remaining: 999ms
7:	learn: 0.3591585	total: 44.3ms	remaining: 968ms
8:	learn: 0.3386794	total: 49.7ms	remaining: 960ms
9:	learn: 0.3307999	total: 56.8ms	remaining: 983ms
10:	learn: 0.3169716	total: 61.2ms	remaining: 958ms
11:	learn: 0.3053442	total: 66.4ms	remaining: 947ms
12:	learn: 0.2919820	total: 72.8ms	remaining: 951ms
13:	learn: 0.2750235	total: 75.9ms	remaining: 916ms
14:	learn: 0.2702600	total: 79.2ms	remaining: 887ms
15:	learn: 0.2665067	total: 82.9ms	remaining: 866ms
16:	learn: 0.2565324	total: 88.8ms	remaining: 867ms
17:	learn: 0.2512708	total: 92.8ms	remaining: 850ms
18:	learn: 0.2456895	total: 96.6ms	remaining: 833ms
19:	learn: 0.2382681	to

C:\Users\user\anaconda3\Lib\site-packages\xgboost\training.py:183: UserWarning: [20:15:56] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


0:	learn: 0.6015724	total: 5.21ms	remaining: 776ms
1:	learn: 0.5642385	total: 10.8ms	remaining: 798ms
2:	learn: 0.5134264	total: 14ms	remaining: 686ms
3:	learn: 0.4886564	total: 17.9ms	remaining: 654ms
4:	learn: 0.4567461	total: 22.5ms	remaining: 654ms
5:	learn: 0.4468541	total: 28.4ms	remaining: 681ms
6:	learn: 0.4082446	total: 32.1ms	remaining: 656ms
7:	learn: 0.3979180	total: 35.6ms	remaining: 633ms
8:	learn: 0.3746062	total: 39.1ms	remaining: 613ms
9:	learn: 0.3677118	total: 44.7ms	remaining: 626ms
10:	learn: 0.3435497	total: 48.6ms	remaining: 615ms
11:	learn: 0.3381786	total: 53.1ms	remaining: 611ms
12:	learn: 0.3198582	total: 57.4ms	remaining: 605ms
13:	learn: 0.3080691	total: 62.3ms	remaining: 605ms
14:	learn: 0.3040418	total: 65.5ms	remaining: 589ms
15:	learn: 0.2959649	total: 69.5ms	remaining: 582ms
16:	learn: 0.2788510	total: 75.7ms	remaining: 592ms
17:	learn: 0.2739593	total: 78.6ms	remaining: 576ms
18:	learn: 0.2721010	total: 82.3ms	remaining: 567ms
19:	learn: 0.2669930	tot

C:\Users\user\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
[I 2025-06-18 20:16:11,856] A new study created in memory with name: no-name-00158753-3d31-418a-be18-bf83f40aa7dd
[I 2025-06-18 20:16:12,741] Trial 0 finished with value: -0.9851870848960775 and parameters: {'C': 0.5611516415334505, 'penalty': 'l1'}. Best is trial 0 with value: -0.9851870848960775.
[I 2025-06-18 20:16:13,423] Trial 1 finished with value: -0.9828118846694796 and parameters: {'C': 1.5751320499779735, 'penalty': 'l1'}. Best is trial 0 with value: -0.9851870848960775.
[I 2025-06-18 20:16:13,602] Trial 2 finished with value: -0.9851558983825598 and parameters: {'C': 0.13066739238053282, 'penalty': 'l1'}. Best is trial 0 with value: -0.9851870848960775.
[I 2025-06-18 20:16:14,017] Trial 3 finished with value: -0.9725302049148304 and parameters: {'C

In [5]:
# --- Сохранение и вывод результатов ---
output_classification_file = Path('classification_results_all_optimizers_50_iter.csv')

all_classification_results_df = pd.DataFrame(all_classification_results)
all_classification_results_df.to_csv(output_classification_file, index=False)
print(f"\nРезультаты классификации сохранены в: {output_classification_file}")

print("\n--- Сводка результатов классификации по методам оптимизации ---")

for optimizer in optimizers:
    print(f"\n## Результаты {optimizer} (Классификация):")
    subset_optimizer = all_classification_results_df[all_classification_results_df['optimizer'] == optimizer]
    print(subset_optimizer.sort_values(by=['target', 'roc_auc'], ascending=[True, False]).to_string())
    print("\n" + "-"*50 + "\n")

# Визуализация метрик классификации (например, ROC-AUC и F1-score)
classification_metrics_to_plot = ['roc_auc', 'f1_score', 'accuracy']

for target_class in classification_targets.keys():
    for metric in classification_metrics_to_plot:
        plt.figure(figsize=(16, 8))
        subset = all_classification_results_df[all_classification_results_df['target'] == target_class].sort_values(by=metric, ascending=False)
        sns.barplot(x='model', y=metric, hue='optimizer', data=subset, palette='viridis')
        plt.title(f'Сравнение {metric.upper()} для "{target_class}" по методам оптимизации', fontsize=16)
        plt.ylabel(metric.upper(), fontsize=12)
        plt.xlabel('Модель', fontsize=12)
        plt.xticks(rotation=45, ha='right', fontsize=10)
        plt.yticks(fontsize=10)
        plt.legend(title='Метод оптимизации', bbox_to_anchor=(1.05, 1), loc='upper left')
        plt.tight_layout()
        plt.savefig(f'classification_{target_class}_{metric}_comparison.png')
        plt.close() # Close plot to free memory


Результаты классификации сохранены в: classification_results_all_optimizers_50_iter.csv

--- Сводка результатов классификации по методам оптимизации ---

## Результаты RandomizedSearchCV (Классификация):
                     model           optimizer                target                                                                                                                              best_params  accuracy  precision  recall  f1_score   roc_auc        data_source
17      CatBoostClassifier  RandomizedSearchCV  is_CC50_above_median                                  {'depth': 4, 'iterations': 183, 'l2_leaf_reg': 2.455591640077322, 'learning_rate': 0.09515504917299872}  0.985075   0.970874    1.00  0.985222  0.997921  Manual Aggregated
3       CatBoostClassifier  RandomizedSearchCV  is_CC50_above_median                                   {'depth': 4, 'iterations': 181, 'l2_leaf_reg': 7.59541228979397, 'learning_rate': 0.09449323267683088}  0.980100   0.961538    1.00  0.980392  0